# Exercise 2 - Create a Docker container image for scoring

## Task 1 - Test locally with Local Web service Compute target

In [ ]:
#REGISTER MODEL TO WORK WITH IT (deploy it)----------
#Connect to your workspace
from azureml.core import Workspace
ws = Workspace.from_config()
from azureml.core.model import Model

#Register your model
model = Model.register(model_path=r'./outputs/models/DecisionTreeClassifier.pkl',
                       model_name="DecisionTreeClassifier",
                       tags={'area': 'covid19', 'type': 'classification'},
                       description='Survival to Covid-19 classification and estimation',
                       workspace=ws)

#Verification of the model
print(model.name, model.id, model.version, sep='\t')
#----------------------------------------------------

#### Script to test the deployment and the model

Note that %%writefile is NOT a Python magic but a command of Jupyter Notebook. Avoid commenting before the %%writefile command, it would break the behavior of this command.

In [ ]:
%%writefile ./sources/scoring_script.py

import json
import pickle as pkl
import pandas as pd
import numpy as np

def init():
    from azureml.core.model import Model
    global model
    model_name = 'DecisionTreeClassifier'
    model_path = Model.get_model_path(model_name=model_name)
    model = pkl.load(open(model_path, 'rb'))

def run(data):
    test = json.loads(data)
    result = model.predict(np.array(test["X_test"]).reshape(-1, 6)).tolist()
    return result

In [ ]:
#Environment
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

env = Environment(name='lab04_environment')
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
env.python.conda_dependencies=conda_dep
#Register your custom environment for further use
env.register(workspace=ws)

In [ ]:
#Inference configuration (docker image, environment of the inference service)
from azureml.core.model import InferenceConfig

lab04_inference_config = InferenceConfig(
    environment=env,
    source_directory='./sources/',
    entry_script='scoring_script.py',
)

In [ ]:
#deployment configuration (machine size, CPU, GPU, i.e. the compute target requested by your Web Service)
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
#Deployment : Genreates a Docker build context
service = Model.deploy(
    ws,
    'covid19-survival-pred-local-test',
    [model],
    lab04_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

#Deployment logs
print(service.get_logs())

Now open the terminal and run the following command :

*curl -X POST -d '{"X_test":\[0, 1, 2, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

Now try with someone way older (change the "2" - age bean 20-30 by an "8" - age bean for the 80+ years old) and you will observe that the odds about survival are not the same.
*curl -X POST -d '{"X_test":\[0, 1, 8, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

*Note*: Just as a reminder, the values passed in the list for the inference are (in this order): \[current_status, sex, age_group, hosp_yn, icu_yn, medcond_yn\]

## Task 2 - Deploy on managed compute and test live

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'akscovidcluster'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

aks_target = AksCompute(ws, aks_name)
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, 'akscovidservice', [model], lab04_inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Now go to the test interface of your newly created real-time endpoint, named "akscovidservice", choose the "Test" tab, and copy / paste the following data in the capture box: *{"X_test":\[0, 1, 2, 1, 0, 1\]}*

*Note*: Just as a reminder, the values passed in the list for the inference are (in this order): \[current_status, sex, age_group, hosp_yn, icu_yn, medcond_yn\]

#### Now try to call your Web Service in Python

BEWARE: You will have to change the api_key with your own one. You can find it when you edit your endpoint and open the "Consume" tab. The api_key is named under "Primary key". You can for security purposes regenerate it. When you do that, you can use the second key temporarily while the primary one is re-generating, in order to not interrupt your service.

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {"X_test":[0, 1, 2, 1, 0, 1]}

body = str.encode(json.dumps(data))

url = 'http://20.75.130.25:80/api/v1/service/akscovidservice/score'
api_key = 'qGnjLJCsoJBlQs6wn01PWQl5oqnF2CjS' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

# Exercise 3 - Deploy custom Docker container image with managed compute

## Task 1 - Deploy custom Docker container image with managed compute

### Define packages versions

You can pin each package versions in a requirements.txt file

Pinning the exact version of the packages you use allows you to run some reproductible experiences, collaborate with others avoiding environment discrepancies, or issues if a library you use releases a new version that breaks your running scripts.

*Note*: In this task, we will not use this requirements file for other purposes than listing our packages versions. The installation of the packages themselves, within the environment, will be managed through a dockerfile in this exercise.
We will use this file in a more concrete manner in the Task 2 of this exercise.

In [ ]:
%%writefile requirements.txt
    azureml-defaults
    azureml.core==1.27.0
    pip==20.1.1
    scikit-learn==0.22.2
    pandas==0.25.3

#### Create a custom Docker container image

*Note* that you can also cherry-pick among the Docker images provided by Azure. We recommend custom Docker images in case you need to install non-Python packages as dependencies

In [ ]:
#WRITE YOUR DOCKER FILE------------------------------
#Note that the dockerfile is here written as a string and not as a file.
#It works as well, and for more permanent, usual and collabotrative process
#we would rather recommend that you write it down as a file containing all
#these steps in a more permanent and "portable" location. Anyway it will be done
#when you will register your environment
dockerfile = r"""
FROM mcr.microsoft.com/mlops/python:latest
#Install the dependancies
RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir azureml-defaults && \
    pip install --no-cache-dir azureml-core==1.27.0 &&\
    pip install --no-cache-dir scikit-learn==0.22.2 && \
    pip install --no-cache-dir pandas==0.25.3
"""

#You have now all you need to create your own docker image within your environment

#----------------------------------------------------

#### Define your environment

In [ ]:
#Environment
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration

env = Environment(name='lab04_environment')
#Creates the environment inside a Docker container
env.docker.enabled = True #Still working but deprecated
#Set base image to None, because the image is defined by dockerfile
env.docker.base_image = None
env.docker.base_dockerfile = dockerfile
#To deactivate Conda and only use the packages you need/want
env.python.user_managed_dependencies=True
#With a custom Docker image you have to specify the inferencing stack version added to the image which is the latest
env.inferencing_stack_version='latest'
#Register your custom environment for further use
env.register(workspace=ws)

In [ ]:
#Inference configuration (docker image, environment of the inference service)
from azureml.core.model import InferenceConfig

lab04_inference_config = InferenceConfig(
    environment=env,
    source_directory='./sources/',
    entry_script='scoring_script.py',
)

In [ ]:
#deployment configuration (machine size, CPU, GPU, i.e. the compute target requested by your Web Service)
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
#Deployment : Genreates a Docker build context
service = Model.deploy(
    ws,
    'covid19-survival-pred-local-test',
    [model],
    lab04_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

print(service.get_logs())

Now open the terminal and run the following command :

*curl -X POST -d '{"X_test":\[0, 1, 2, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

Now try with someone way older (change the "2" - age bean 20-30 by an "8" - age bean for the 80+ years old) and you will observe that the odds about survival are not the same.
*curl -X POST -d '{"X_test":\[0, 1, 8, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

*Note*: Just as a reminder, the values passed in the list for the inference are (in this order): \[current_status, sex, age_group, hosp_yn, icu_yn, medcond_yn\]

## Task 2 - Pinning packages versions with requirements.txt

#### Let's use requirements.txt to pin your packages versions, build and deploy your environment

In [ ]:
#Environment using requirements.txt
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_pip_requirements(name='lab04_environment', file_path='requirements.txt')
env.register(workspace=ws)
lab04_inference_config = InferenceConfig(
    environment=env,
    source_directory='./sources/',
    entry_script='scoring_script.py',
)
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)
service = Model.deploy(
    ws,
    'covid19-survival-pred-local-test',
    [model],
    lab04_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Now open the terminal and run the following command :

*curl -X POST -d '{"X_test":\[0, 1, 2, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

Now try with someone way older (change the "2" - age bean 20-30 by an "8" - age bean for the 80+ years old) and you will observe that the odds about survival are not the same.
*curl -X POST -d '{"X_test":\[0, 1, 8, 1, 0, 1\]}' -H "Content-Type: application/json" http://localhost:6789/score*

*Note*: Just as a reminder, the values passed in the list for the inference are (in this order): \[current_status, sex, age_group, hosp_yn, icu_yn, medcond_yn\]